In [ ]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("jinaai/jina-embeddings-v3", trust_remote_code=True)

In [ ]:
model.encode('').shape

In [ ]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./data.db")

guide_collection_name = "guides"
query_collection_name = "samples"

In [ ]:
if milvus_client.has_collection(guide_collection_name):
    milvus_client.drop_collection(guide_collection_name)

if milvus_client.has_collection(query_collection_name):
    milvus_client.drop_collection(query_collection_name)


In [ ]:
from pymilvus import FieldSchema, CollectionSchema, DataType

sample_fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=1024),
    FieldSchema(name="description", dtype=DataType.VARCHAR, max_length=1000),
    FieldSchema(name="query", dtype=DataType.VARCHAR, max_length=8000),
    FieldSchema(name="tables", dtype=DataType.JSON),  # Changed to JSON type for dict storage
    FieldSchema(name="lang", dtype=DataType.VARCHAR, max_length=20)
]

guide_fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=1024),
    FieldSchema(name="description", dtype=DataType.VARCHAR, max_length=8000),
    FieldSchema(name="keyword", dtype=DataType.VARCHAR, max_length=500),
    FieldSchema(name="lang", dtype=DataType.VARCHAR, max_length=20)
]


In [ ]:
sample_schema = CollectionSchema(
    fields=sample_fields,
    description="Collection for storing document embeddings"
)

guide_schema = CollectionSchema(
    fields=guide_fields,
    description="Collection for storing document embeddings"
)

In [ ]:
milvus_client.create_collection(
    collection_name=query_collection_name,
    schema=sample_schema
)

milvus_client.create_collection(
    collection_name=guide_collection_name,
    schema=guide_schema
)

In [ ]:
index_params = MilvusClient.prepare_index_params()

index_params.add_index(
    field_name="vector",
    metric_type="COSINE",
    index_type="IVF_FLAT",
    index_name="vector_index",
    params={ "nlist": 128 }
)

milvus_client.create_index(
    collection_name=query_collection_name,
    index_params=index_params,
)

milvus_client.create_index(
    collection_name=guide_collection_name,
    index_params=index_params,
)


In [ ]:
from tqdm import tqdm
import json

with open('./samples.json', 'r') as f:
    text_lines = json.load(f)

data = []

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append(
        {
            "vector": model.encode(str(line["description"])).tolist(),
            "description": line["description"],
            "query": line["query"],
            "tables": line["tables"],
            "lang": line["lang"]
        }
    )

milvus_client.insert(collection_name=query_collection_name, data=data)

In [ ]:
import json
from tqdm import tqdm

with open('./keywords.json', 'r') as f:
    text_lines = json.load(f)

data = []

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    vector = (
        model.encode(f"{line['keyword']} có nghĩa là {line['description']}").tolist()
        if line["lang"] == "vi"
        else model.encode(f"{line['keyword']} means {line['description']}").tolist()
    )
    data.append(
        {
            "vector": vector,
            "description": line["description"],
            "keyword": line["keyword"],
            "lang": line["lang"],
        }
    )

milvus_client.insert(collection_name=guide_collection_name, data=data)

In [ ]:
# question = "Calculate the total number of fresh embryo transfers and the total number of fresh embryo transfers with positive βHCG for each month in 2024, then calculate the ratio of positive βHCG to the total number of fresh embryo transfers."
# question = "Lấy PID của tất cả bệnh nhân thực hiện thủ thuật chọc hút trong năm 2024"
question = "Bệnh nhân với code 24861508 có tên là gì. Họ có bao nhiêu mẫu trữ ở bệnh viện, bao nhiêu cái hết hạn và còn hạn?"
# question = "Có bao nhiêu bệnh nhân hết hạn mẫu trữ loại phôi trong tháng 2 năm nay"
# question = "Có bao nhiêu bệnh nhân đang điều trị FET hoặc ICSI/IVF."
# question = "Liệt kê tổng số ca chuyển phôi trữ trong các tháng năm 2024"
# question = "Liệt kê tổng số ca thai sinh hóa trong chuyển phôi tươi của các tháng trong năm 2024"
# question = "Tính tổng số ca chuyển phôi tươi của các tháng trong năm 2024"
# question = "Tính tổng số ca chuyển phôi tươi và tổng số ca có BHCG (+) chuyển phôi tươi của các tháng trong năm 2024, sau đó tỉnh tỉ lệ BHCG (+) trên tổng số chuyển phôi tươi"

In [ ]:
search_res = milvus_client.search(
    collection_name=guide_collection_name,
    data=[
        model.encode(question).tolist()
    ],  # Use the `emb_text` function to convert the question to an embedding vector
    limit=5,  # Return top 3 results
    filter="lang == 'vi'",  # Filter
    search_params={"metric_type": "COSINE", "params": {}},  # Inner product distance
    output_fields=["description", "keyword"],  # Return the text field
)

In [ ]:
import json
retrieved_lines_with_distances = [
    (res["entity"]["keyword"], res["entity"]["description"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4, ensure_ascii=False))

In [ ]:
search_res = milvus_client.search(
    collection_name=query_collection_name,
    data=[
        model.encode(question).tolist()
    ],  # Use the `emb_text` function to convert the question to an embedding vector
    limit=3,  # Return top 3 results
    filter="lang == 'vi'",  # Filter
    search_params={"metric_type": "COSINE", "params": {}},  # Inner product distance
    output_fields=["description"],  # Return the text field
)

In [ ]:
import json
retrieved_lines_with_distances = [
    (res["entity"]["description"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4, ensure_ascii=False))

In [ ]:
search_res = milvus_client.query(
    collection_name="tables",
    filter="lang == 'vi'",
    output_fields=["description", "name"],
)

In [ ]:
search_res

In [ ]:
usage_table = [
    "action_update_appointment_follicle_note: Cập nhật số lượng nang trứng và ghi chú cho một thủ thuật y tế.",
    "add_cryostraw_to_renew: Wizard chọn ống đông lạnh cần gia hạn.",
    "check_sperm_donate_history: Wizard kiểm tra lịch sử hiến tinh của bệnh nhân qua thông tin định danh.",
    "history_medical_treatment_iui: Lưu trữ lịch sử điều trị IUI",
    "history_medical_treatment_ivf: Lưu trữ thông tin điều trị IVF",
    "hr_contract_type: Quản lý các loại hợp đồng lao động",
    "hr_department: Quản lý các phòng ban trong công ty",
    "hr_departure_reason: Quản lý lý do nghỉ việc của nhân viên",
    "hr_employee: Quản lý thông tin nhân viên",
    "hr_employee_category: Quản lý thẻ (tags) gán cho nhân viên",
    "hr_job: Quản lý các vị trí công việc trong công ty",
    "medical_appointment_check_foetus: Quản lý khám thai định kỳ",
    "medical_appointment_procedure: Quản lý thủ thuật y tế",
    "medical_appointment_re_examination: Quản lý tái khám bệnh nhân",
    "medical_beta_hcg_pregnant: Quản lý xét nghiệm beta HCG và theo dõi thai kỳ",
    "medical_birth_info: Quản lý thông tin em bé sau điều trị",
    "medical_blood_type: Quản lý danh mục nhóm máu",
    "medical_bottle_medium: Quản lý lọ môi trường nuôi cấy trong phòng lab IVF",
    "medical_canes: Quản lý ống đựng mẫu sinh học trong hệ thống bảo quản lạnh",
    "medical_canes_position: Quản lý vị trí lưu trữ mẫu sinh học trong ống đựng",
    "medical_canister: Quản lý thông tin của canister trong hệ thống bảo quản lạnh",
    "medical_canister_layer: Quản lý các tầng trong canister",
    "medical_canister_position: Quản lý từng vị trí cụ thể trong canister",
    "medical_cassette: Quản lý thông tin về cassette trong hệ thống bảo quản mẫu sinh học",
    "medical_cause_infertility: Danh mục nguyên nhân vô sinh",
    "medical_create_culture: Tạo và cập nhật thông tin nuôi cấy phôi/trứng",
    "medical_create_relation_popup: Tạo và quản lý mối quan hệ giữa các bệnh nhân",
    "medical_cryopreserv_straw: Quản lý bảo quản lạnh các mẫu sinh học",
    "medical_day_of_embryo: Quản lý thông tin về các ngày phát triển của phôi",
    "medical_department: Quản lý các khoa/phòng trong bệnh viện hoặc trung tâm y tế",
    "medical_detail_embryo_frozen: Theo dõi chi tiết các phôi đã đông lạnh",
    "medical_diagnostic: Quản lý danh mục các chẩn đoán y tế",
    "medical_document_type: Quản lý danh mục các loại tài liệu trong hệ thống y tế",
    "medical_embryo_biopsy_detail: Lưu trữ thông tin chi tiết về mẫu sinh thiết phôi",
    "medical_embryo_culture: Theo dõi và quản lý quá trình nuôi cấy phôi",
    "medical_embryo_detail: Thống kê số lượng phôi theo ngày",
    "medical_embryo_grading: Phân loại chất lượng phôi",
    "medical_embryonic_stage: Quản lý các giai đoạn phát triển của phôi",
    "medical_embryonic_stage_condition: Định nghĩa điều kiện đánh giá phôi",
    "medical_ex_husband: Quản lý thông tin tiền sử sinh sản",
    "medical_follicular_monitoring_decision: Quản lý quyết định theo dõi nang noãn",
    "medical_follicular_size: Lưu trữ thông tin kích thước nang noãn",
    "medical_follow_follicular: Quản lý theo dõi nang noãn",
    "medical_group_embryo: Quản lý nhóm đông lạnh phôi/trứng",
    "medical_history_embryo_culture: Ghi nhận lịch sử nuôi cấy phôi",
    "medical_history_relationship: Quản lý lịch sử mối quan hệ bệnh nhân",
    "medical_hr_department_line: Liên kết nhân viên và bộ phận",
    "medical_icd: Quản lý danh mục bệnh tật quốc tế",
    "medical_image_embryo: Quản lý hình ảnh phôi",
    "medical_indication: Quản lý chỉ định xét nghiệm y tế",
    "medical_indication_detail: Chi tiết chỉ định xét nghiệm",
    "medical_indication_detail_select: Lựa chọn chỉ số xét nghiệm",
    "medical_indication_reference: Tham chiếu chỉ định xét nghiệm",
    "medical_indication_trigger: Quản lý trigger cho chỉ định xét nghiệm",
    "medical_initial_evaluation: Đánh giá ban đầu bệnh nhân",
    "medical_initial_evaluation_prescription: Đơn thuốc đánh giá ban đầu",
    "medical_location: Vị trí lưu trữ mẫu sinh học",
    "medical_location_room: Thông tin phòng lưu trữ mẫu vật y tế",
    "medical_outpatient_examination: Khám bệnh ngoại trú",
    "medical_patient: Thông tin cơ bản bệnh nhân",
    "medical_patient_action_reference_bank: Cập nhật mã tham chiếu ngân hàng mô",
    "medical_patient_documen: Quản lý giấy tờ bệnh nhân",
    "medical_patient_document_submit: Theo dõi nộp giấy tờ bệnh nhân",
    "medical_patient_partner: Thông tin người thân/người đại diện bệnh nhân",
    "medical_patient_printed_report_event: Lịch sử in báo cáo bệnh nhân",
    "medical_patient_record: Hồ sơ bệnh án",
    "medical_patient_relationship: Mối quan hệ điều trị giữa bệnh nhân",
    "medical_patient_report_wizard_line: Quản lý in báo cáo y tế",
    "medical_pgs_info: Thông tin quy trình PGS",
    "medical_pgs_info_line: chi tiết của quy trình PGS",
    "medical_prepare_medium: chuẩn bị môi trường nuôi cấy",
    "medical_prepare_medium_detail: chi tiết chuẩn bị môi trường nuôi cấy",
    "medical_prescription_order: quản lý đơn thuốc y tế",
    "medical_prescription_order_line: quản lý chi tiết đơn thuốc",
    "medical_prescription_usage: quản lý cách dùng thuốc",
    "medical_procedure_room: quản lý thông tin về phòng thủ thuật",
    "medical_process_pgt: quản lý quy trình PGT",
    "medical_process_storage: quản lý quy trình lưu trữ phôi/noãn",
    "medical_process_transfer: quản lý quá trình chuyển phôi thai",
    "medical_protocol: quản lý các quy trình y tế",
    "medical_record_document: định nghĩa cấu trúc hồ sơ y tế",
    "medical_record_type: quản lý các loại hồ sơ y tế",
    "medical_renewal: quản lý việc gia hạn thời gian lưu trữ mẫu y tế",
    "medical_renewal_line: theo dõi chi tiết việc gia hạn lưu trữ mẫu sinh học",
    "medical_renewal_reminder: quản lý việc nhắc nhở gia hạn lưu trữ mẫu sinh học",
    "medical_renewal_reminder_log: ghi nhận lịch sử các hoạt động nhắc nhở gia hạn",
    "medical_renewal_reminder_log_line: ghi lại chi tiết các hoạt động nhắc nhở gia hạn",
    "medical_renewal_reminder_queue: quản lý hàng đợi nhắc nhở gia hạn dịch vụ y tế",
    "medical_storage_configuration: cấu hình các thông số cho việc lưu trữ và nhắc nhở",
    "medical_storage_history: theo dõi lịch sử lưu trữ mẫu sinh học",
    "medical_storage_process: quản lý quy trình lưu trữ các mẫu sinh học",
    "medical_storage_tool: Quản lý dụng cụ và thiết bị lưu trữ mẫu sinh học",
    "medical_tank: Quản lý bình chứa lạnh",
    "medical_tank_information_record: Ghi nhận thông tin theo dõi bình chứa",
    "medical_test: Quản lý phiếu xét nghiệm y tế",
    "medical_test_category: Phân loại xét nghiệm y tế",
    "medical_test_group: Nhóm xét nghiệm y tế",
    "medical_test_indices: Định nghĩa chỉ số xét nghiệm",
    "medical_test_indices_result_selection: Định nghĩa lựa chọn kết quả xét nghiệm",
    "medical_test_result: Quản lý kết quả xét nghiệm",
    "medical_thaw_bank_popup: Quản lý quy trình rã đông mẫu sinh học",
    "medical_thaw_popup: Quản lý quy trình rã đông mẫu sinh học",
    "medical_thaw_popup_detail: Chi tiết quy trình rã đông mẫu sinh học",
    "medical_thawing_process: Quản lý quy trình rã đông mẫu sinh học",
    "medical_transfer_bank: Chuyển mẫu vật y tế vào ngân hàng lưu trữ",
    "medical_treatment: Quản lý quy trình điều trị y tế",
    "medical_treatment_document: Quản lý tài liệu điều trị y tế",
    "medical_treatment_medium: Quản lý môi trường nuôi cấy điều trị y tế",
    "medical_treatment_result: Quản lý kết quả điều trị y tế",
    "medical_treatment_result_beta: Theo dõi kết quả xét nghiệm beta HCG",
    "medical_treatment_stage: Quản lý giai đoạn điều trị y tế",
    "medical_treatment_technique: Quản lý kỹ thuật điều trị y tế",
    "medical_update_relation_popup: Cập nhật thông tin mối quan hệ bệnh nhân",
    "medical_waiting_procedure: Quản lý hàng đợi thủ tục y tế",
    "medium_type: Quản lý loại môi trường nuôi cấy",
    "patient_fingerprint: Lưu trữ thông tin vân tay bệnh nhân",
    "patient_fingerprint_history: Lịch sử xác thực vân tay bệnh nhân",
    "patient_recognition_history: Lịch sử nhận diện bệnh nhân",
    "process_storage_line: Lưu trữ quá trình cấp đông mẫu",
    "process_storage_popup: Quản lý lưu trữ mẫu y sinh học",
    "product_attribute: Quản lý thuộc tính sản phẩm",
    "product_attribute_custom_value: Lưu trữ giá trị tùy chỉnh thuộc tính sản phẩm",
    "product_attribute_value: Lưu trữ giá trị thuộc tính sản phẩm",
    "product_category: Quản lý danh mục sản phẩm",
    "product_label_layout: Tạo và in nhãn sản phẩm",
    "product_packaging: Quản lý đóng gói sản phẩm",
    "product_pricelist: Quản lý bảng giá sản phẩm",
    "product_pricelist_item: Định nghĩa quy tắc tính giá sản phẩm",
    "product_product: Quản lý biến thể sản phẩm",
    "product_supplierinfo: Quản lý bảng giá sản phẩm từ nhà cung cấp",
    "product_tag: Quản lý thẻ sản phẩm",
    "product_template: Quản lý thông tin sản phẩm",
    "product_template_attribute_exclusion: Định nghĩa quy tắc loại trừ thuộc tính sản phẩm",
    "product_template_attribute_line: Quản lý thuộc tính sản phẩm",
    "product_template_attribute_value: Lưu trữ mối quan hệ thuộc tính sản phẩm",
    "range_years_old: Định nghĩa khoảng tuổi",
    "relation_contact: Quản lý thông tin mối quan hệ tạm thời",
    "renew_cryostraw_group: Nhóm ống đông lạnh cần gia hạn",
    "res_partner: Quản lý thông tin nhân viên",
    "res_users: Quản lý người dùng hệ thống",
    "treatment_test_subscriber: Đăng ký theo dõi xét nghiệm điều trị",
    "uom_category: Quản lý đơn vị đo lường sản phẩm",
    "uom_uom: Phân loại đơn vị đo lường"
]


In [ ]:
usage_table = [
    "action_update_appointment_follicle_note: là wizard dùng để cập nhật số lượng nang trứng (follicle) và ghi chú cho một thủ thuật y tế. Nó cho phép nhập số lượng nang trứng trước và sau khi thu nhận, trạng thái (+/-), cùng ghi chú bổ sung.",
    "add_cryostraw_to_renew: là wizard dùng để chọn các ống đông lạnh (cryostraw) cần gia hạn. Nó cho phép người dùng chọn theo nhóm các ống đông lạnh (phôi, noãn, tinh trùng) để đưa vào đơn gia hạn, kèm theo thông tin về thời hạn lưu trữ và ngày hết hạn.",
    "check_sperm_donate_history: là wizard dùng để kiểm tra lịch sử hiến tinh của bệnh nhân. Nó cho phép tìm kiếm bệnh nhân thông qua các thông tin định danh (mã bệnh nhân, CCCD, hộ chiếu, SĐT) và trả về kết quả về việc bệnh nhân có phải là người hiến tinh hay không, đồng thời cho phép mở hồ sơ bệnh nhân nếu tìm thấy.",
    "history_medical_treatment_iui: là model dùng để lưu trữ lịch sử điều trị bằng phương pháp IUI (bơm tinh trùng vào buồng tử cung). Nó ghi lại thông tin về phác đồ điều trị, liều FSH, ngày thực hiện IUI và kết quả điều trị của một ca điều trị y tế.",
    "history_medical_treatment_ivf: là model lưu trữ thông tin chi tiết về lịch sử điều trị IVF của bệnh nhân, bao gồm các dữ liệu về phác đồ, chọc hút trứng, phát triển phôi và kết quả điều trị.",
    "hr_contract_type: là model quản lý các loại hợp đồng lao động trong hệ thống quản lý nhân sự, cho phép định nghĩa và sắp xếp các kiểu hợp đồng khác nhau.",
    "hr_department: dùng để quản lý các phòng ban trong công ty, bao gồm thông tin về tên phòng ban, người quản lý, nhân viên, phòng ban cha-con và các kế hoạch nhân sự liên quan.",
    "hr_departure_reason: dùng để quản lý các lý do nghỉ việc của nhân viên, bao gồm các lý do mặc định như sa thải (fired), từ chức (resigned), và nghỉ hưu (retired). Model này không cho phép xóa các lý do nghỉ việc mặc định.",
    "hr_employee: là model chính để quản lý thông tin nhân viên trong công ty, bao gồm các thông tin cá nhân (tên, ngày sinh, địa chỉ...), thông tin công việc (phòng ban, vị trí, người quản lý...)",
    "hr_employee_category: dùng để quản lý các thẻ (tags) gán cho nhân viên, mỗi thẻ có tên duy nhất và màu sắc ngẫu nhiên. Thẻ này giúp phân loại và nhóm nhân viên theo các tiêu chí khác nhau.",
    "hr_job: dùng để quản lý các vị trí công việc trong công ty, bao gồm thông tin về tên vị trí, số lượng nhân viên hiện tại, số lượng cần tuyển dụng, mô tả công việc và yêu cầu.",
    "medical_appointment_check_foetus: là mô hình quản lý các lần khám thai định kỳ",
    "medical_appointment_procedure: là model quản lý các thủ thuật y tế (chọc hút trứng, chuyển phôi, IUI, rã đông, sinh thiết phôi), bao gồm thông tin về bệnh nhân, bác sĩ thực hiện, thời gian hẹn và kết quả thủ thuật.",
    "medical_appointment_re_examination: là model quản lý các cuộc tái khám cho bệnh nhân, bao gồm thông tin về vợ/chồng, ghi chú tái khám, loại tái khám (như phẫu thuật tinh hoàn, lấy trứng, chuyển phôi tươi/đông...) và ngày tái khám.",
    "medical_beta_hcg_pregnant: là model quản lý xét nghiệm beta HCG và theo dõi thai kỳ sau các phương pháp điều trị IVF/FET/IUI, bao gồm kết quả xét nghiệm beta HCG, tình trạng thai (thai hóa học/lâm sàng/ngoài tử cung), thông tin em bé khi sinh (số lượng/cân nặng/giới tính), phương pháp sinh và các dữ liệu siêu âm liên quan",
    "medical_birth_info: là model quản lý thông tin chi tiết về em bé được sinh ra sau điều trị, bao gồm mô tả, giới tính và cân nặng của từng em bé, liên kết với kết quả xét nghiệm beta HCG và kết quả điều trị.",
    "medical_blood_type: là model quản lý danh mục các loại nhóm máu (ví dụ: A, B, O, AB), bao gồm mã code, tên gọi và thứ tự sắp xếp của nhóm máu.",
    "medical_bottle_medium: là model quản lý các lọ môi trường nuôi cấy trong phòng lab IVF, bao gồm thông tin về tên môi trường, số lô (lot), mã vạch, ngày sản xuất, ngày mở, hạn sử dụng, ngày chuẩn bị và tình trạng sử dụng. Model này giúp theo dõi và quản lý việc sử dụng môi trường nuôi cấy trong quy trình điều trị IVF.",
    "medical_canes: là model quản lý các ống đựng mẫu sinh học trong hệ thống bảo quản lạnh (cryo-container), được sử dụng để lưu trữ các mẫu như trứng, phôi, tinh trùng trong quy trình hỗ trợ sinh sản.",
    "medical_canes_position: quản lý vị trí lưu trữ của mẫu sinh học trong một ống đựng (cane) thuộc hệ thống bảo quản lạnh, bao gồm các thông tin về mẫu sinh học, bệnh nhân, trạng thái và thời gian lưu trữ",
    "medical_canister: Quản lý thông tin của canister (giỏ/thùng đựng) trong hệ thống bảo quản lạnh, bao gồm số tầng, số vị trí trên mỗi tầng, và theo dõi tình trạng sử dụng của canister trong bình nitơ.",
    "medical_canister_layer: quản lý các tầng trong canister (giỏ/thùng đựng) của hệ thống bảo quản lạnh, mỗi tầng được đánh số thứ tự và theo dõi số lượng vị trí còn trống có thể sử dụng để lưu trữ mẫu sinh học.",
    "medical_canister_position: Quản lý từng vị trí cụ thể trong canister, mỗi vị trí có thể chứa một ống đựng mẫu (cane), theo dõi thông tin như bệnh nhân, số lượng mẫu đang lưu trữ tại vị trí đó.",
    "medical_cassette: là model quản lý thông tin về cassette (hộp đựng) trong hệ thống bảo quản mẫu sinh học, được kế thừa và mở rộng để thêm thông tin về thiết bị y tế (labware) liên quan đến cassette đó.",
    "medical_cause_infertility: là bảng danh mục dùng để lưu trữ các nguyên nhân vô sinh.",
    "medical_create_culture: hỗ trợ người dùng tạo và cập nhật thông tin nuôi cấy phôi/trứng trong quy trình IVF thông qua giao diện wizard có cấu trúc, bao gồm các thông số về trứng, môi trường nuôi cấy, và theo dõi quá trình xử lý.",
    "medical_create_relation_popup: là một wizard hỗ trợ tạo và quản lý mối quan hệ giữa các bệnh nhân trong quy trình điều trị IVF, cho phép thiết lập các vai trò như vợ-chồng, người cho trứng/tinh trùng, người mang thai hộ và kiểm soát tính hợp lệ của các mối quan hệ thông qua giao diện popup.",
    "medical_cryopreserv_straw: quản lý việc bảo quản lạnh (cryopreservation) các mẫu sinh học (như trứng, phôi, tinh trùng) và theo dõi các nhắc nhở gia hạn thời gian lưu trữ cho bệnh nhân.",
    "medical_day_of_embryo: Quản lý thông tin về các ngày phát triển của phôi, bao gồm tên gọi, mã code và thứ tự của từng ngày. Model này dùng để theo dõi và phân loại các giai đoạn phát triển của phôi trong quá trình điều trị.",
    "medical_department: dùng để quản lý các khoa/phòng trong bệnh viện hoặc trung tâm y tế, bao gồm thông tin như mã tham chiếu, tên khoa, người quản lý và đơn vị trực thuộc.",
    "medical_detail_embryo_frozen: dùng để theo dõi chi tiết các phôi đã đông lạnh trong một đợt điều trị, bao gồm thông tin về ngày (giai đoạn phát triển), số lượng phôi và danh sách các phôi được nuôi cấy (cultures).",
    "medical_diagnostic: dùng để quản lý danh mục các chẩn đoán y tế, bao gồm thông tin như mã tham chiếu, tên chẩn đoán (có hỗ trợ đa ngôn ngữ) và thứ tự hiển thị.",
    "medical_document_type: Quản lý danh mục các loại tài liệu trong hệ thống y tế, bao gồm thông tin cơ bản như tên tài liệu, mô tả và số lượng cần thiết cho mỗi loại.",
    "medical_embryo_biopsy_detail: là một bảng lưu trữ thông tin chi tiết về mẫu sinh thiết phôi trong quá trình điều trị, bao gồm tên mẫu, kết quả xét nghiệm, các ghi chú và tệp đính kèm.",
    "medical_embryo_culture: là một bảng quan trọng dùng để theo dõi và quản lý quá trình nuôi cấy phôi từ ngày 0 đến ngày 7, bao gồm các thông tin như giai đoạn phát triển của phôi, hình thái phôi, quyết định xử lý (đông lạnh, nuôi cấy tiếp, chuyển phôi), thông tin sinh thiết nếu có.",
    "medical_embryo_detail: là bảng thống kê số lượng phôi theo từng ngày (từ ngày 2 đến ngày 8), bao gồm số lượng phôi phân loại theo chất lượng (Grade 1-4), tốc độ phát triển (chậm, bình thường, nhanh, ngừng phát triển) và quyết định xử lý (đông lạnh, nuôi cấy tiếp, hủy bỏ, chuyển phôi, sinh thiết).",
    "medical_embryo_grading: Đây là model dùng để cấu hình và phân loại chất lượng phôi, với hệ thống mã code và tên gọi riêng, cho phép sắp xếp theo thứ tự ưu tiên thông qua trường sequence.",
    "medical_embryonic_stage: Đây là model quản lý các giai đoạn phát triển của phôi, có khả năng theo dõi thay đổi (mail.thread) và tính toán đánh giá phôi dựa trên các điều kiện được cấu hình.",
    "medical_embryonic_stage_condition: Đây là model dùng để định nghĩa các điều kiện đánh giá cho từng giai đoạn phát triển của phôi.",
    "medical_ex_husband: Model quản lý thông tin tiền sử sinh sản của bệnh nhân với chồng cũ, bao gồm các thông tin như số lần mang thai, số con, số lần sẩy thai, phá thai, thai lưu và các biến chứng thai nghén.",
    "medical_follicular_monitoring_decision: Model quản lý các quyết định theo dõi nang noãn trong quá trình điều trị, cho phép cấu hình theo kỹ thuật điều trị khác nhau và phân biệt giữa người cho/nhận trứng, được sắp xếp theo thứ tự ưu tiên.",
    "medical_follicular_size: Model lưu trữ thông tin kích thước và số lượng nang noãn ở hai bên buồng trứng (trái/phải) trong quá trình theo dõi nang noãn, được sắp xếp theo kích thước giảm dần.",
    "medical_follow_follicular: Model quản lý và theo dõi quá trình phát triển nang noãn trong điều trị IVF/ICSI, bao gồm các thông tin về kích thước nang noãn hai bên buồng trứng, chỉ định điều trị, lịch hẹn và quyết định điều trị cho từng giai đoạn.",
    "medical_group_embryo: Model quản lý nhóm đông lạnh phôi/trứng, với mã code unique và thứ tự sắp xếp ưu tiên, được sử dụng để phân loại và theo dõi các nhóm phôi/trứng trong quá trình bảo quản đông lạnh.",
    "medical_history_embryo_culture: Model ghi nhận lịch sử quá trình nuôi cấy phôi theo thời gian, bao gồm các thông tin về mẫu phôi, người thực hiện và thời gian cho các giai đoạn: nguồn gốc, lưu trữ, rã đông và chuyển phôi.",
    "medical_history_relationship: Model quản lý lịch sử các mối quan hệ của bệnh nhân, có khả năng tự động tạo mã tham chiếu (sequence) và liên kết với thông tin quan hệ bệnh nhân thông qua bảng medical_patient_relationship",
    "medical_hr_department_line: Model liên kết giữa nhân viên và bộ phận, cho phép thiết lập tên bộ phận (đa ngôn ngữ), mã code và quản lý danh sách nhân viên thuộc bộ phận thông qua quan hệ many2many.",
    "medical_icd: Model quản lý danh mục bệnh tật quốc tế (ICD - International Classification of Diseases), với thông tin mã bệnh và tên bệnh bằng tiếng Việt và tiếng Anh, hỗ trợ tìm kiếm theo mã hoặc tên bệnh.",
    "medical_image_embryo: được sử dụng để lưu trữ và quản lý các hình ảnh phôi trong quá trình điều trị y tế, cho phép sắp xếp thứ tự và khả năng phóng to ảnh.",
    "medical_indication: là hệ thống quản lý chỉ định xét nghiệm y tế, bao gồm thông tin bệnh nhân, bác sĩ, các xét nghiệm được chỉ định, tính toán chi phí và theo dõi trạng thái của chỉ định.",
    "medical_indication_detail: là chi tiết của chỉ định xét nghiệm, quản lý thông tin về các chỉ số xét nghiệm cụ thể, giá tiền, giảm giá và tính toán chi phí cho từng xét nghiệm trong một chỉ định y tế.",
    "medical_indication_detail_select: là một bảng tạm thời để lựa chọn các chỉ số xét nghiệm trước khi thêm vào bảng chi tiết chỉ định chính (medical_indication_detail).",
    "medical_indication_reference: là bảng lưu trữ các tham chiếu liên kết giữa chi tiết chỉ định với các tài liệu/bản ghi khác trong hệ thống",
    "medical_indication_trigger: là module quản lý các trigger (bộ kích hoạt) cho chỉ định xét nghiệm, cho phép thực thi các hành động tự động dựa trên điều kiện được định nghĩa thông qua Python code, được sử dụng để liên kết với các model khác trong hệ thống.",
    "medical_initial_evaluation: là module quản lý đánh giá ban đầu cho bệnh nhân.",
    "medical_initial_evaluation_prescription: là module quản lý thông tin về đơn thuốc trong đánh giá ban đầu, bao gồm tên thuốc, liều dùng, cách dùng và thời gian sử dụng thuốc cuối cùng.",
    "medical_location: là module quản lý vị trí lưu trữ mẫu sinh học trong phòng lab.",
    "medical_location_room: là module quản lý thông tin cơ bản của phòng lưu trữ mẫu vật y tế như tên phòng, kích thước và các chỉ số môi trường (nhiệt độ, PM, AQI).",
    "medical_outpatient_examination: là module quản lý thông tin khám bệnh ngoại trú, bao gồm thông tin bệnh nhân, các chỉ số sinh hiệu (mạch, huyết áp, nhiệt độ...), dịch vụ khám và lý do khám bệnh.",
    "medical_patient: là module quản lý thông tin cơ bản của bệnh nhân như mã số, họ tên, ngày sinh, giới tính và các hồ sơ khám bệnh liên quan.",
    "medical_patient_action_reference_bank: à wizard dùng để cập nhật mã tham chiếu ngân hàng mô cho bệnh nhân, kiểm tra các điều kiện (mẫu rã đông, tồn kho) trước khi cho phép cập nhật mã mới.",
    "medical_patient_documen: là module quản lý và theo dõi giấy tờ của bệnh nhân, bao gồm việc kiểm soát số lượng giấy tờ cần nộp, đã nộp và tính toán tỷ lệ hoàn thành hồ sơ theo từng loại.",
    "medical_patient_document_submit: là module theo dõi chi tiết việc nộp từng loại giấy tờ của bệnh nhân, ghi nhận số lượng đã nộp, file đính kèm và thời hạn hiệu lực của giấy tờ.",
    "medical_patient_partner: là module quản lý thông tin người thân/người đại diện của bệnh nhân, với các thông tin cơ bản như tên, ngày sinh, giới tính và mối quan hệ với bệnh nhân (vợ/chồng/con/người đại diện).",
    "medical_patient_printed_report_event: là module ghi nhận lịch sử in ấn các báo cáo của bệnh nhân, bao gồm thông tin về người in, loại báo cáo và các mã QR/barcode liên quan của bệnh nhân và người thân.",
    "medical_patient_record: là module quản lý hồ sơ bệnh án, lưu trữ thông tin y tế và tiền sử của bệnh nhân với 3 trạng thái: Mới/Đã duyệt/Đã hủy. Module này có quan hệ Many2one với bảng medical_patient",
    "medical_patient_relationship: là module quản lý mối quan hệ điều trị giữa các bệnh nhân (vợ/chồng/người hiến), đảm bảo tính hợp lệ của mối quan hệ dựa trên vai trò và giới tính, và theo dõi lịch sử thay đổi các mối quan hệ.",
    "medical_patient_report_wizard_line: quản lý việc in ấn và theo dõi lịch sử báo cáo y tế cho bệnh nhân, bao gồm thông tin về thời gian in, số lần in và các đối tượng liên quan.",
    "medical_pgs_info: Quản lý thông tin về quy trình PGS (Pre-implantation Genetic Screening) trong thụ tinh ống nghiệm, bao gồm các dữ liệu về ngày rã đông phôi và lịch trình điều trị IVF.",
    "medical_pgs_info_line: chi tiết của quy trình PGS, lưu trữ thông tin về ngày rã đông phôi cụ thể và số ngày lịch trình tương ứng, là một bản ghi con của medical_pgs_info.",
    "medical_prepare_medium: dùng để chuẩn bị môi trường nuôi cấy và quản lý thông tin cho các thủ thuật y tế liên quan đến thu trứng (OR - Oocyte Retrieval) và chuyển phôi đông lạnh (FET - Frozen Embryo Transfer).",
    "medical_prepare_medium_detail: dùng để lưu trữ và quản lý chi tiết việc chuẩn bị môi trường nuôi cấy trong quy trình y tế, bao gồm thông tin về bệnh nhân, thủ tục hẹn khám, và các chỉ định liên quan.",
    "medical_prescription_order: quản lý đơn thuốc y tế bao gồm thông tin bệnh nhân, bác sĩ kê đơn, chi tiết thuốc (sáng/trưa/chiều/tối), liên kết với điều trị và theo dõi nang noãn.",
    "medical_prescription_order_line: quản lý chi tiết đơn thuốc, bao gồm thông tin về thuốc, liều dùng (sáng/trưa/chiều/tối), số ngày uống, cách dùng (trước/sau ăn), đơn vị tính và hướng dẫn sử dụng.",
    "medical_prescription_usage: quản lý cách dùng thuốc trong đơn thuốc, với hai thông tin chính là mã tham chiếu (code) và tên cách dùng (name), hỗ trợ đa ngôn ngữ.",
    "medical_procedure_room: dùng để quản lý thông tin về phòng thủ thuật trong cơ sở y tế, bao gồm tên phòng (name) và mã phòng (code).",
    "medical_process_pgt: dùng để xử lý và quản lý quy trình PGT (Preimplantation Genetic Testing) trong điều trị IVF, bao gồm việc theo dõi mẫu sinh thiết phôi, ngày sinh thiết và liên kết với quá trình nuôi cấy phôi.",
    "medical_process_storage: quản lý quy trình lưu trữ phôi/noãn trong điều trị IVF, bao gồm việc sinh mã tự động cho phôi được sinh thiết (biopsy_code) và mã đông lạnh (freezing_code) cho phôi/noãn, đồng thời theo dõi trạng thái lưu trữ của mẫu sinh học.",
    "medical_process_transfer: dùng để xử lý quá trình chuyển phôi thai (embryo transfer) trong điều trị y tế, cho phép người dùng chọn các mẫu phôi để chuyển và cập nhật thông tin liên quan đến quá trình chuyển phôi vào hồ sơ điều trị.",
    "medical_protocol: quản lý các quy trình y tế với mã tham chiếu, tên, mô tả và liên kết với các kỹ thuật điều trị tương ứng.",
    "medical_record_document: Định nghĩa cấu trúc hồ sơ y tế bằng cách kết nối loại tài liệu với loại hồ sơ, cho phép cấu hình tính bắt buộc và thứ tự sắp xếp của từng tài liệu trong hồ sơ.",
    "medical_record_type: quản lý các loại hồ sơ y tế với tên, mã tham chiếu và danh sách tài liệu y tế liên quan thông qua quan hệ One2many với model medical_record_document.",
    "medical_renewal: quản lý việc gia hạn thời gian lưu trữ các mẫu y tế (phôi, noãn, tinh trùng), bao gồm thông tin về bệnh nhân, loại mẫu, thời gian gia hạn và trạng thái lưu trữ của mẫu.",
    "medical_renewal_line: theo dõi chi tiết việc gia hạn lưu trữ mẫu sinh học bao gồm ngày gia hạn, thời gian gia hạn và kiểu gia hạn (kỳ hạn/mới/hủy).",
    "medical_renewal_reminder: quản lý việc nhắc nhở gia hạn lưu trữ mẫu sinh học (phôi, noãn, tinh trùng), có các chức năng gửi email/SMS thông báo, tạo hoạt động nhắc nhở (activity) và theo dõi lịch sử nhắc nhở cho từng bệnh nhân.",
    "medical_renewal_reminder_log: ghi nhận lịch sử các hoạt động nhắc nhở gia hạn (qua email/SMS/điện thoại) cho mẫu sinh học, bao gồm loại nhắc nhở, thời gian, người thực hiện và chi tiết nội dung nhắc.",
    "medical_renewal_reminder_log_line: Model này dùng để ghi lại chi tiết các hoạt động nhắc nhở gia hạn.",
    "medical_renewal_reminder_queue: Model này dùng để quản lý hàng đợi nhắc nhở gia hạn dịch vụ y tế, theo dõi trạng thái của các thông báo nhắc nhở (ready/assigned/done) và xử lý việc gửi thông báo qua các kênh khác nhau.",
    "medical_storage_configuration: Model này dùng để cấu hình các thông số cho việc lưu trữ và nhắc nhở các mẫu sinh học (oocyte, phôi, tinh trùng)",
    "medical_storage_history: Model này dùng để theo dõi lịch sử lưu trữ mẫu sinh học (phôi, noãn, tinh trùng) trong hệ thống ngân hàng lạnh",
    "medical_storage_process: Model quản lý quy trình lưu trữ các mẫu sinh học trong phòng khám/bệnh viện như: noãn (oocyte), hợp tử (zygote), phôi (embryo), tinh trùng (sperm). Model bao gồm các thông tin về bệnh nhân, thời gian đông lạnh, lý do đông lạnh, vị trí lưu trữ và các thông số sinh học liên quan.",
    "medical_storage_tool: Model quản lý các dụng cụ và thiết bị lưu trữ mẫu sinh học (Cryo-storage), bao gồm thông tin về tên thiết bị, nhà cung cấp, số vị trí lưu trữ, và loại mẫu có thể lưu trữ (noãn-phôi/tinh trùng/cả hai).",
    "medical_tank: Model quản lý bình chứa lạnh (tank) dùng để bảo quản mẫu sinh học, bao gồm thông tin về vị trí, dung lượng, nhiệt độ và trạng thái sử dụng của bình.",
    "medical_tank_information_record: Model dùng để ghi nhận thông tin theo dõi định kỳ của bình chứa (tank) trong hệ thống lưu trữ lạnh, bao gồm các chỉ số quan trọng như nhiệt độ, thể tích và mức chất lỏng.",
    "medical_test: mô tả một phiếu xét nghiệm y tế, lưu trữ thông tin về bệnh nhân, bác sĩ, kết quả xét nghiệm và theo dõi trạng thái từ lúc tiếp nhận đến khi hoàn thành.",
    "medical_test_category:  dùng để phân loại các xét nghiệm y tế, với mỗi loại xét nghiệm có mã riêng (code), chỉ số đo (indices) và thứ tự ưu tiên (sequence) để sắp xếp hiển thị.",
    "medical_test_group: dùng để nhóm các chỉ số xét nghiệm (indices) theo từng loại xét nghiệm (test_category_id), giúp tổ chức và quản lý các chỉ số xét nghiệm một cách có hệ thống.",
    "medical_test_indices: định nghĩa các chỉ số/thông số của một xét nghiệm y tế, bao gồm tên, đơn vị đo, khoảng tham chiếu và cách hiển thị/tính toán kết quả.",
    "medical_test_indices_result_selection: dùng để định nghĩa các lựa chọn kết quả cho những chỉ số xét nghiệm dạng lựa chọn (radio, dropdown, checkbox), mỗi lựa chọn có mã code riêng và thứ tự hiển thị.",
    "medical_test_result: là một mô hình quản lý kết quả xét nghiệm y tế, lưu trữ các chỉ số xét nghiệm cụ thể (như huyết áp, đường huyết...) cùng với các thông tin liên quan như đơn vị đo, giá trị tham chiếu cho nam/nữ.",
    "medical_thaw_bank_popup: là một wizard để quản lý và thực hiện quy trình rã đông mẫu sinh học (tinh trùng, phôi, trứng) từ ngân hàng mô theo các tiêu chí đặc thù như nhóm máu, chiều cao, màu da và trình độ học vấn của người hiến tặng.",
    "medical_thaw_popup: là một wizard để quản lý quy trình rã đông các mẫu sinh học (phôi, trứng, tinh trùng) với các thông tin chi tiết như thời gian rã đông, số lượng mẫu, giai đoạn phôi và cách thức theo dõi quá trình rã đông.",
    "medical_thaw_popup_detail: là một model chi tiết của wizard rã đông, dùng để lưu trữ thông tin chi tiết của từng ống mẫu sinh học cần rã đông như: thông tin bệnh nhân, vị trí lưu trữ, nồng độ, hình thái phôi và trạng thái của mẫu.",
    "medical_thawing_process: à một mô hình quản lý quy trình rã đông các mẫu sinh học (tinh trùng, noãn, phôi...) trong điều trị vô sinh, bao gồm thông tin về bệnh nhân, thời gian, lý do rã đông, vị trí lưu trữ và theo dõi trạng thái của quy trình từ lúc bắt đầu đến khi hoàn thành.",
    "medical_transfer_bank: là một wizard dùng để chuyển mẫu vật y tế (như tinh trùng) vào ngân hàng lưu trữ, cho phép người dùng lựa chọn cách chuyển (tạo vị trí mới hoặc giữ vị trí cũ) và tự động tạo mã lưu trữ cho mỗi mẫu.",
    "medical_treatment: là một mô hình quản lý quy trình điều trị y tế, theo dõi toàn bộ quá trình điều trị của bệnh nhân bao gồm: thông tin bệnh nhân, phương pháp điều trị (IVF, IUI, FET...), lịch trình thực hiện các thủ thuật, thông tin lấy trứng, tinh trùng, phôi và kết quả điều trị.",
    "medical_treatment_document: là model dùng để lưu trữ các tài liệu/hồ sơ đính kèm trong quá trình điều trị của bệnh nhân, cho phép upload file và ghi chú mô tả cho từng tài liệu.",
    "medical_treatment_medium: là một mô hình quản lý các môi trường nuôi cấy được sử dụng trong điều trị y tế, cho phép liên kết với các liệu pháp điều trị, xác định loại môi trường nuôi cấy, các kỹ thuật xử lý và theo dõi theo giới tính của bệnh nhân.",
    "medical_treatment_result: là mô hình quản lý kết quả điều trị hiếm muộn, theo dõi các chỉ số beta HCG, thông tin thai kỳ, kết quả sinh, biến chứng và lịch hẹn siêu âm của bệnh nhân.",
    "medical_treatment_result_beta: là một mô hình theo dõi kết quả xét nghiệm beta HCG trong quá trình điều trị vô sinh",
    "medical_treatment_stage: là mô hình quản lý các giai đoạn điều trị y tế, bao gồm thông tin như tên giai đoạn, mã tham chiếu, thứ tự, kỹ thuật điều trị, và các thuộc tính khác như giới tính áp dụng và trạng thái hiển thị trên thanh trạng thái.",
    "medical_treatment_technique: là một mô hình quản lý các kỹ thuật điều trị y tế, bao gồm mã tham chiếu, tên kỹ thuật, các giai đoạn điều trị liên quan và một trình tự số tham chiếu tự động được tạo theo định dạng cụ thể.",
    "medical_update_relation_popup: dùng để cập nhật thông tin mối quan hệ giữa các bệnh nhân, cho phép thiết lập vai trò (như vợ/chồng/người hiến tinh trùng...) và kiểm tra các ràng buộc về giới tính, quan hệ giữa các bệnh nhân.",
    "medical_waiting_procedure:  là một mô hình quản lý hàng đợi thủ tục y tế, theo dõi danh sách bệnh nhân đang chờ thực hiện các thủ tục với bác sĩ được chỉ định. Model này hỗ trợ việc quản lý trạng thái của thủ tục (mới/đang mở/hoàn thành/hủy) và tích hợp chức năng gửi thông báo tự động.",
    "medium_type: là một mô hình quản lý các loại môi trường nuôi cấy trong xét nghiệm y tế, bao gồm tên, mã tham chiếu, kỹ thuật áp dụng và giới tính áp dụng.",
    "patient_fingerprint: là mô hình lưu trữ thông tin vân tay của bệnh nhân, bao gồm dữ liệu ảnh vân tay và template vân tay cho từng ngón tay cụ thể, được sử dụng để nhận dạng và xác thực danh tính bệnh nhân.",
    "patient_fingerprint_history: để lưu trữ lịch sử xác thực vân tay của bệnh nhân, ghi lại thông tin về thời gian kiểm tra, kết quả (thành công/thất bại), điểm số xác thực, và hình ảnh vân tay.",
    "patient_recognition_history: là mô hình lưu trữ lịch sử nhận diện bệnh nhân, ghi lại thông tin về hình ảnh nhận diện, độ chính xác nhận diện, và cảnh báo trường hợp có bệnh nhân trùng lặp hoặc tương tự trong hệ thống.",
    "process_storage_line: dùng để lưu trữ chi tiết quá trình cấp đông mẫu trong phòng lab, bao gồm thông tin về phôi/noãn được nuôi cấy (culture), mã culture, phân loại chất lượng (grade), đặc điểm hiện tại và thuộc nhóm cấp đông nào.",
    "process_storage_popup: dùng để xử lý quá trình lưu trữ mẫu y sinh học (phôi, noãn, tinh trùng) vào các thiết bị trữ lạnh như bình nitơ, với các thông tin về vị trí lưu trữ (tank, canister, canes), thông tin về mẫu và người thực hiện.",
    "product_attribute: dùng để quản lý các thuộc tính của sản phẩm (như màu sắc, kích thước...), giúp tạo ra các biến thể sản phẩm khác nhau dựa trên tổ hợp các giá trị thuộc tính.",
    "product_attribute_custom_value: dùng để lưu trữ các giá trị tùy chỉnh của thuộc tính sản phẩm, cho phép người dùng nhập giá trị riêng thay vì chỉ chọn từ các giá trị có sẵn.",
    "product_attribute_value: dùng để lưu trữ các giá trị tùy chỉnh cho thuộc tính sản phẩm",
    "product_category: dùng để quản lý các danh mục sản phẩm theo cấu trúc cây phân cấp (cha-con), cho phép phân nhóm và tổ chức sản phẩm thành các danh mục khác nhau.",
    "product_label_layout: dùng để tạo và in nhãn sản phẩm với các định dạng khác nhau (Dymo, 2x7, 4x7, 4x12), cho phép người dùng tùy chỉnh số lượng và bố cục của nhãn sản phẩm trên trang in.",
    "product_packaging: dùng để quản lý đóng gói sản phẩm, bao gồm thông tin về tên đóng gói, số lượng chứa trong một đơn vị đóng gói, mã vạch",
    "product_pricelist: dùng để quản lý các bảng giá sản phẩm, cho phép thiết lập các quy tắc tính giá khác nhau dựa trên nhiều tiêu chí như số lượng, đơn vị tính, ngày áp dụng, loại sản phẩm và tiền tệ.",
    "product_pricelist_item: dùng để định nghĩa các quy tắc tính giá sản phẩm trong bảng giá, cho phép thiết lập giá cố định, tỷ lệ giảm giá hoặc công thức tính giá dựa trên nhiều tiêu chí như danh mục sản phẩm, số lượng tối thiểu, thời gian áp dụng.",
    "product_product: dùng để quản lý các biến thể (variants) của sản phẩm, kế thừa từ model product.template. Model này chứa các thông tin chi tiết như giá bán, mã vạch, hình ảnh và các thuộc tính riêng của từng biến thể sản phẩm.",
    "product_supplierinfo: dùng để quản lý bảng giá sản phẩm từ nhà cung cấp, bao gồm thông tin về giá cả, số lượng tối thiểu, thời gian giao hàng và các điều khoản mua hàng từ nhà cung cấp cụ thể.",
    "product_tag: dùng để quản lý các thẻ (tag) gán cho sản phẩm trong Odoo, cho phép phân loại và tìm kiếm sản phẩm dựa trên tag.",
    "product_template: dùng để quản lý thông tin sản phẩm/dịch vụ trong hệ thống, bao gồm tên, giá bán, đơn vị tính, thuộc tính sản phẩm, biến thể sản phẩm và các cấu hình liên quan.",
    "product_template_attribute_exclusion: dùng để định nghĩa các quy tắc loại trừ giữa các thuộc tính sản phẩm, cho phép thiết lập việc một giá trị thuộc tính nào đó không thể kết hợp với một số giá trị thuộc tính khác trên cùng sản phẩm.",
    "product_template_attribute_line: dùng để quản lý các thuộc tính có sẵn trên một sản phẩm template, bao gồm các giá trị thuộc tính được chọn trong một quan hệ many-to-many. Model này đóng vai trò như một mô hình cấu hình để tạo ra các giá trị thuộc tính sản phẩm tương ứng.",
    "product_template_attribute_value: dùng để lưu trữ mối quan hệ giữa các giá trị thuộc tính và sản phẩm mẫu (product template).",
    "range_years_old: là một mô hình đơn giản để định nghĩa các khoảng tuổi, với các trường thông tin như mô tả (name), thứ tự sắp xếp (sequence), tuổi bắt đầu (from_age) và tuổi kết thúc (to_age).",
    "relation_contact:  dùng để quản lý thông tin về mối quan hệ tạm thời giữa các bệnh nhân, định nghĩa vai trò của từng người (vợ, chồng, người cho trứng/tinh trùng...), kiểm tra tính hợp lệ của vai trò dựa trên giới tính và đảm bảo một số vai trò chỉ được xuất hiện một lần trong một mối quan hệ.",
    "renew_cryostraw_group: là mô hình tạm thời (TransientModel) dùng để nhóm các ống đông lạnh (cryostraw) cần gia hạn",
    "res_partner: được kế thừa để bổ sung thêm các chức năng liên quan đến quản lý nhân viên, cho phép lưu trữ thông tin địa chỉ cá nhân của nhân viên và theo dõi số lượng nhân viên liên quan đến mỗi partner.",
    "res_users: là model quản lý người dùng trong hệ thống Odoo, bao gồm thông tin đăng nhập, phân quyền truy cập, thiết lập cá nhân và các khóa API cho người dùng.",
    "treatment_test_subscriber: là một mô hình dùng để đăng ký theo dõi xét nghiệm trong quá trình điều trị, cho phép định nghĩa các chỉ số xét nghiệm cần theo dõi (indices_ids), loại kỹ thuật điều trị (technique_id), và khoảng thời gian theo dõi (date_shift).",
    "uom_category: dùng để quản lý các đơn vị đo lường của sản phẩm trong hệ thống",
    "uom_uom: dùng để phân loại các đơn vị đo lường (UoM - Unit of Measure), cho phép nhóm các đơn vị đo có thể chuyển đổi qua lại với nhau (ví dụ: kg, g, mg trong cùng một nhóm đơn vị đo khối lượng). Mỗi nhóm phải có một đơn vị tham chiếu (reference unit) làm cơ sở để chuyển đổi.",
]

In [ ]:
question = """Tìm cho tôi danh sách bệnh nhân chọc hút của năm 2024, lấy các trường sau:
- Họ tên bệnh nhân
- Mã bệnh nhân PID
- Ngày sinh
- Ngày thủ thuật
- chỉ số BMI của bệnh nhân
- Kết quả xét nghiệm FSH gần nhất so với ngày thủ thuật"""
question = "Có bao nhiêu bệnh nhân điều trị FET hoặc ICSI/IVF trong năm 2024?"